First, import the library.

We will also need to have LaTeX installed on your system for rendering the diagrams. We also import official Q# library as we will use it for comparison of the generated diagrams.

In [ ]:
from qsharp_bridge import *
from IPython.core.getipython import get_ipython
from IPython.core.interactiveshell import InteractiveShell
from qsharp_widgets import Circuit
import qsharp

# Ensure that TeX binaries are in the PATH for LaTeX rendering
os.environ["PATH"] += os.pathsep + "/Library/TeX/texbin"

Next, let's load the Jupyter TikZ extension. 

In [ ]:
%load_ext jupyter_tikz

Now we can write some Q# code. This will be the Q# code for which we will generate a circuit diagram. We will set up two test code snippets - one that consists of a single operation, and another that has an operation calling another operation.

In [ ]:
code_1 = """
operation Main() : Result[] {
    use qubits = Qubit[8];

    // apply Hadamard gate to the first qubit
    H(qubits[0]);

    // apply CNOT gates to create entanglement
    for qubit in qubits[1..Length(qubits) - 1] {
        CNOT(qubits[0], qubit);
    }

    // return measurement results
    MResetEachZ(qubits)
}
"""

code_2 = """
namespace MyQuantumApp {
    @EntryPoint()
    operation Run() : (Result, Result) {
        use (control, target) = (Qubit(), Qubit());
        PrepareBellState(control, target);
        
        let resultControl = MResetZ(control);
        let resultTarget = MResetZ(target);
        return (resultControl, resultTarget);
    }

    operation PrepareBellState(q1 : Qubit, q2: Qubit) : Unit {
        H(q1);
        CNOT(q1, q2);
    }
}
"""

Once the Q# code is written, we can use the `quantikz` function to generate and display the circuit diagram directly in the Jupyter notebook.

In [ ]:
quantikz_diagram_1 = quantikz(code_1, options=QuantikzGenerationOptions(group_by_scope=False))
quantikz_diagram_2 = quantikz(code_2, options=QuantikzGenerationOptions(group_by_scope=False))

# for debugging, display the generated LaTeX code
print(quantikz_diagram_1)
print(quantikz_diagram_2)

We can now use the TikZ extension to render a PDF from the LaTeX code generated by Q# Bridge to verify the output.

In [ ]:
shell = get_ipython()
assert shell is not None

shell.run_cell_magic('tikz', '-l quantikz', quantikz_diagram_1)

In [ ]:
shell = get_ipython()
assert shell is not None

shell.run_cell_magic('tikz', '-l quantikz', quantikz_diagram_2)

Notice how the two-operation sample was actually decomposed into its constituent individual gates in the generated diagram - this is because we set `group_by_scope` to `False` in the `QuantikzGenerationOptions` when generating the diagrams.

For comparison, use the built-in `Circuit` widget from the `qsharp_widgets` library to display the same circuit diagram. This should give us the same visual representation of the quantum circuit.

In [ ]:
qsharp.init() # this ensures clean state
qsharp.eval(code_1)
Circuit(qsharp.circuit("Main()"))  

In [ ]:

qsharp.init() # this ensures clean state
qsharp.eval(code_2)
Circuit(qsharp.circuit("MyQuantumApp.Run()")) 

We should see that the diagrams generated using `quantikz` and the built-in `Circuit` widget are identical. This confirms that our LaTeX generation is working correctly!

Q# compiler (and, by extension, Q# Bridge) can also group gates on the circuit into groups scoped by an operation. In our case that would mean a large single block representing `PrepareBellState`.

Let's set `group_by_scope` of the `QuantikzGenerationOptions` to `True` and pass that into the `quantikz` function - and see how the generated diagram changes.

In [ ]:
quantikz_diagram_3 = quantikz(code_2, options=QuantikzGenerationOptions(group_by_scope=True))

# for debugging, display the generated LaTeX code
print(quantikz_diagram_3)

shell = get_ipython()
assert shell is not None

shell.run_cell_magic('tikz', '-l quantikz', quantikz_diagram_3)

Congratualations! You have successfully generated quantum circuit diagrams from Q# code using Q# Bridge and LaTeX in a Jupyter notebook.